# Search Work and Export BibTeX

In [ ]:
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import database
from snowballing.operations import load_work, work_to_bibtex, reload
from snowballing import config
reload()

In [ ]:
def find(text):
    words = text.split()
    for work in load_work():
        match = True
        for word in words:
            if not any(word.lower() in str(getattr(work, attr)).lower() for attr in dir(work) if not attr.startswith("_")):
                match = False
                break
        if match:
            yield work_to_bibtex(work)

In [ ]:
from ipywidgets import widgets, interactive

def result(text):
    if len(text) > 2:
        for work in find(text):
            print(work.replace("\n ", "\n  "))
interactive(result, text="")

# Match BibTeX file to database work

In [ ]:
bibtex_file = 'bibliography.bib'

from snowballing.operations import match_bibtex_to_work
with open(bibtex_file) as bibtex_file:
    bibtex_str = bibtex_file.read()

matched = match_bibtex_to_work(bibtex_str.split("%Entries")[-1])
works = dict(map(reversed, matched))

### Check if all snowballed approaches appear in the bibtex

In [ ]:
from snowballing.approaches import get_approaches
all_approaches = get_approaches()
len(script)

In [ ]:
for a, m in all_approaches:
    for w in a.work:
        if w not in works and "snowball" in w.category:
            print(w @ metakey, w)

### Look for unmatched work

In [ ]:
from copy import copy
from snowballing.operations import info_to_code, bibtex_to_info
unmatched = [x[0] for x in matched if not x[1]]
for unmatch in unmatched:
    print(info_to_code(bibtex_to_info(copy(unmatch))))

### Recreate Bibtex

In [ ]:
from snowballing.operations import work_to_bibtex_entry
result = [
    work_to_bibtex_entry(work, name=entry['ID'], acronym=True)
    for entry, work in matched
]

In [ ]:
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase
db = BibDatabase()
db.entries = result

writer = BibTexWriter()
writer.indent = '  '     # indent entries with 4 spaces instead of one

In [ ]:
with open(bibtex_file, 'w') as bibtex_file:
    bibtex_file.write(bibtex_str.split("%Entries")[0] + "%Entries\n\n" + writer.write(db))